In [1]:
source("helper.R")
max_expid<-get_max("Log")+1
dataset_name="SPY"
exp_name<-glue("Modelling_{dataset_name}")
log_dir<-glue("Log/{max_expid}.{exp_name}")
logger=get_logger(log_dir, log_name = "log", debug = TRUE)
futile.logger::flog.info("start experiment", name = "log")







if (dataset_name=="China"){
dataset <- load_data("data/IFclose.csv")

}else if (dataset_name=="Saudi"){
dataset <- load_data("data/1180_vol.csv")

}else if (dataset_name=="SPY"){

dataset <- load_data("data/SPY.csv")

}else{

    stop("Error: Invalid data set")
}

mesh <- fm_mesh_1d(dataset$times)

dataset=dataset[(length(dataset$logreturn)-999):length(dataset$logreturn),]

par(bg = "white") 
dim(dataset)


Warning message:
“package ‘INLA’ was built under R version 4.5.1”
Loading required package: Matrix



This is INLA_25.10.01 built 2025-10-01 17:17:12 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


This is rSPDE 2.5.1
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.



INFO [2025-10-07 17:05:46] start experiment


Rows: 3687 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): act_symbol
dbl  (5): open, high, low, close, volume
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 1000    4

In [2]:
dataset$logreturn<-inla.mdata(dataset$logreturn)


# 5 models

In [3]:

OU_model <- rspde.matern(mesh = mesh,parameterization="matern")
formula_OU <-  logreturn ~ -1 + Intercept(1)+ field(times, model = OU_model)
fit<-bru(formula_OU, family="stochvol", data = dataset)
result_fit<- rspde.result(fit, "field", OU_model, parameterization="matern")
print(fit$mlik[2])
summary(result_fit)


flog.info("Model: stochvol",name="log")
summary_df <- as.data.frame(summary(result_fit))
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)


[1] -43370.13


,mean,sd,0.025quant,0.5quant,0.975quant,mode
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
std.dev,0.0223721,0.0253595,0.000996398,0.0141949,0.0915619,0.00235678
range,0.2123620,0.1251660,0.043668400,0.1890700,0.5155270,0.12301300
nu,1.0015200,0.5605470,0.079337500,1.0094900,1.9100700,1.70928000


INFO [2025-10-07 17:05:55] Model: stochvol
INFO [2025-10-07 17:05:55] likelihood:  -43370.1332709885
INFO [2025-10-07 17:05:55] 
             mean        sd  0.025quant  0.5quant 0.975quant       mode
std.dev 0.0223721 0.0253595 0.000996398 0.0141949  0.0915619 0.00235678
range   0.2123620 0.1251660 0.043668400 0.1890700  0.5155270 0.12301300
nu      1.0015200 0.5605470 0.079337500 1.0094900  1.9100700 1.70928000


In [5]:
cloglike <- inla.cloglike.define(model = "inla_cloglike_stochvol",
shlib = "./cloglikelihood/gaussiantest.so",
debug = FALSE)
spde_model <- rspde.matern(mesh = mesh,parameterization="matern")
formula <-  logreturn ~ -1 + Intercept(1)+ field(times, model = spde_model)
fit<-bru(formula, data = dataset,family = "cloglike", control.family = list(cloglike = cloglike))
result_fit<- rspde.result(fit, "field", spde_model, parameterization="matern")
print(fit$mlik[2])
summary(result_fit)


flog.info("Model: stochvol",name="log")
summary_df <- as.data.frame(summary(result_fit))
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)


[1] 3164.049


,mean,sd,0.025quant,0.5quant,0.975quant,mode
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
std.dev,1.608550,0.04177060,1.536710,1.604670,1.699470,1.591950
range,611.698000,24.69880000,561.288000,612.707000,657.827000,616.969000
nu,0.502648,0.00356561,0.497682,0.502014,0.511012,0.499229


INFO [2025-10-07 17:07:33] Model: stochvol
INFO [2025-10-07 17:07:33] likelihood:  3164.0486639261
INFO [2025-10-07 17:07:33] 
              mean          sd 0.025quant   0.5quant 0.975quant       mode
std.dev   1.608550  0.04177060   1.536710   1.604670   1.699470   1.591950
range   611.698000 24.69880000 561.288000 612.707000 657.827000 616.969000
nu        0.502648  0.00356561   0.497682   0.502014   0.511012   0.499229


In [ ]:

OU_model <- rspde.matern(mesh = mesh,parameterization="matern")
formula_OU <- logreturn ~ -1 + Intercept(1)+ field(times, model = OU_model)
fit<-bru(formula_OU, family="stochvol.t", data = dataset)
result_fit<- rspde.result(fit, "field", OU_model, parameterization="matern")
print(fit$mlik[2])
summary(result_fit)

flog.info("Model: stochvol.t",name="log")
summary_df <- as.data.frame(summary(result_fit))
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)


In [ ]:
fit

In [ ]:

OU_model <- rspde.matern(mesh = mesh,parameterization="matern")
formula_OU <- logreturn ~ -1 + Intercept(1)+ field(times, model = OU_model)
fit<-bru(formula_OU, family="stochvol.nig", data = dataset)
result_fit<- rspde.result(fit, "field", OU_model, parameterization="matern")
print(fit$mlik[2])
summary(result_fit)

flog.info("Model: stochvol.nig",name="log")
summary_df <- as.data.frame(summary(result_fit))
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)


In [ ]:

OU_model <- rspde.matern(mesh = mesh,parameterization="matern")
formula_OU <- logreturn ~ -1 + Intercept(1)+ field(times, model = OU_model)
fit<-bru(formula_OU, family="stochvolln", data = dataset)
result_fit<- rspde.result(fit, "field", OU_model, parameterization="matern")
print(fit$mlik[2])
summary(result_fit)

flog.info("Model: lognormal",name="log")
summary_df <- as.data.frame(summary(result_fit))
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)


In [ ]:
rspde_model1 <- rspde.matern(mesh = mesh,nu.upper.bound=1)

rspde_model2 <- rspde.matern(mesh = mesh,nu.upper.bound=2)
formula <- logreturn ~ -1 + Intercept(1) + rough(times, model = rspde_model1) + smooth(times, model = rspde_model2)+f(times,model="iid")

fit <- bru(formula, family="stochvol", data = dataset)
result_fit1<- rspde.result(fit, "rough", rspde_model1, parameterization="matern")
result_fit2<- rspde.result(fit, "smooth", rspde_model2, parameterization="matern")
print(fit$mlik[2])
rbind(summary(result_fit1),summary(result_fit2))

summary_df <- rbind(summary(result_fit1),summary(result_fit2))
flog.info("Model: rough + smooth",name="log")
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)


In [ ]:
summary_df <- rbind(summary(result_fit1),summary(result_fit2))
flog.info("Model: rough + smooth",name="log")
flog.info(paste("likelihood: ", fit$mlik[2]),name="log")
flog.info(
  paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
  name = "log"
)
